# Working with `PointGroupTemplate`s

1. Run the [Initializations](#Initializations)
1. If you have questions, see the [Basics](./Basics.ipynb)

To logically group points, similar to how the Google Digital Buildings project does, we use the concept of a `PointGroupTemplate`.

The best way to learn how a PGT should function is to define a valid PGT dictionary.  Below we define a valid PGT dictionary.  Go through the [EntityTemplate](./EntityTemplates.ipynb) workbook to understand the expected structure for the information defined under the `telemetry_point_types` key.  Else, just follow the pattern.

In [2]:
pgt_dict = {
    'id': '4aa753fc-ab1b-47d0-984f-121fa0cfa0e9',
    'symbol': 'SD',
    'description': 'Single duct VAV type, with basic airflow control.',
    'template_type': 'point-group-template',
    'schema_name': 'Haystack',
    'version': '3.9.9',
    'telemetry_point_types': {
        'discharge-air-flow-sensor-point-cur-his': {
            'curVal': {
                '_kind': 'number',
                'val': None
            },
            'unit': {
                '_kind': 'str',
                'val': 'cfm'
            }
        },
        'discharge-air-flow-sp-point': {},
        'damper-cmd-point': {
            'unit': 'percent'
        }
    }
}

## Creating a `PointGroupTemplate`

We will now create a PGT from the above data structure.  It will resolve as a valid PGT since all of the correct keys are defined. We also demonstrate what an invalid PGT will look like.

In [3]:
valid_pgt = tt.PointGroupTemplate(pgt_dict)
print(f"Valid: {valid_pgt.is_valid}")

invalid_pgt = tt.PointGroupTemplate({})
print(f"Valid: {invalid_pgt.is_valid}")

Valid: True
Valid: False


## Populating a PointGroupTemplate

Only when we have a valid PGT can we populate the basic information and telemetry points for the PGT.  Resolving the telemetry point types creates (in the background) an EntityTemplate for each point type declared (i.e. key in `telemetry_point_types`).

In [ ]:
valid_pgt.populate_template_basics()
valid_pgt.resolve_telemetry_point_types()

# Initializations

In [ ]:
# imports and setup
import os

import tasty.templates as tt
import tasty.constants as tc
import tasty.graphs as tg

haystack_ont = tg.load_ontology('Haystack', '3.9.9')
brick_ont = tg.load_ontology('Brick', '1.1')

point_type_string = 'cur-writable-motor-run-curVal-sensor-point'
haystack_namespaced_terms = tt.get_namespaced_terms(haystack_ont, point_type_string)

brick_type = 'Damper_Position_Command'
brick_namespaced_terms = tt.get_namespaced_terms(brick_ont, brick_type)

fields = {
    'curVal': {
        '_kind': 'number',
        'val': None
    },
    'unit': 'cfm'
}

namespaced_fields = tt.get_namespaced_terms(haystack_ont, fields)
structured_terms = tt.hget_entity_classes(haystack_ont, haystack_namespaced_terms)

# For Brick, we don't have any additional 'typing properties' or fields, so we initialize those as empty sets
bet = tt.EntityTemplate(entity_classes=brick_namespaced_terms,
                        typing_properties=set(),
                        fields=set())

# For Haystack
het = tt.EntityTemplate(entity_classes=structured_terms['classes'],
                        typing_properties=structured_terms['markers'],
                        # use namespaced_fields from earlier
                        fields=namespaced_fields)